# Vehicle Detection with YOLO

This notebook demonstrates vehicle detection and tracking using YOLO11. We'll process a video to detect and track vehicles (cars, buses, trucks, motorcycles, bicycles, and trains).


## Step 1: Install Required Library

Install the Ultralytics library which provides the YOLO model implementation.


In [1]:
!pip install -q numpy==1.26.4 scipy==1.11.4 ultralytics opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 49.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 29.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Check GPU Availability

Verify if CUDA (GPU) is available for faster processing. This will help determine if we can use GPU acceleration for model inference.


In [2]:
import torch
print(torch.cuda.is_available())

True


## Step 3: Import Required Libraries

Import the necessary libraries:
- `cv2`: OpenCV for video processing and visualization
- `YOLO`: Ultralytics YOLO model for object detection and tracking
- `defaultdict`: For tracking vehicle counts (if needed)


In [3]:
import cv2
from ultralytics import YOLO
from collections import defaultdict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Step 4: Load YOLO Model

Load the pre-trained YOLO11 Large model. This model is capable of detecting and tracking multiple object classes including vehicles.


In [4]:
model = YOLO('yolo11l.pt')

## Step 5: View Available Classes

Display the list of all classes that the YOLO model can detect. This helps us understand which class IDs correspond to vehicles (e.g., car=2, bus=5, truck=7).


In [5]:
class_list = model.names
class_list

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

## Step 6: Open Video File

Open the input video file for processing. Make sure to update the path to match your video file location.


In [6]:
cap = cv2.VideoCapture('/kaggle/input/test-video-for-vehicle-counting/Test Video for Vehicle Counting Model.mp4')

## Step 7: Video Processing Setup

Configure logging and visualization parameters for the video processing pipeline.


In [7]:
import cv2
import logging

# 🔇 Silence Ultralytics logs
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# ---------------- CONFIG ----------------
font_scale_label = 0.45
box_thickness = 2
# ----------------------------------------

class_list = model.names

# Video writer
save_output = True
if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(
        "output.mp4",
        fourcc,
        30,
        (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
         int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    )

# ---------------- MAIN LOOP ----------------
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        persist=True,
        classes=[1, 2, 3, 5, 6, 7],
        verbose=False
    )

    if results[0].boxes is not None and results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_indices = results[0].boxes.cls.int().cpu().tolist()

        for box, track_id, class_idx in zip(boxes, track_ids, class_indices):
            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2

            class_name = class_list[class_idx]

            # Bounding box
            cv2.rectangle(
                frame,
                (x1, y1),
                (x2, y2),
                (0, 200, 0),
                box_thickness
            )

            # Center point
            cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)

            # Label
            cv2.putText(
                frame,
                f"{class_name} | ID {track_id}",
                (x1, max(y1 - 6, 15)),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale_label,
                (255, 255, 255),
                1
            )

    if save_output:
        out.write(frame)

# ---------------- CLEANUP ----------------
cap.release()
if save_output:
    out.release()

print("✅ Video processed successfully.")

✅ Video processed successfully.
